<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [109]:


import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [110]:
# if not (shutil.which("glpk") or os.path.isfile("glpk")):
#     if "google.colab" in sys.modules:
#         !apt-get install -y -qq glpk-utils
#     else:
#         try:
#             !conda install -c conda-forge glpk
#         except:
#             pass

In [111]:
!git clone https://github.com/DarleneJD/ACOPF.git

fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [40]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# Solver (HiGHS)
from pyomo.environ import *
import pandas as pd
import numpy as np


In [41]:
base_path = "/content/ACOPF"


# MT

In [ ]:
branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

df_branches = pd.read_excel(branches_path, sheet_name="MT")
df_trafos   = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses    = pd.read_excel(buses_path,    sheet_name="MT")


In [ ]:
def normalize_phases(value):
    """
    Converte entradas como:
    1
    1.0
    '1,2,3'
    '1.0,2.0,3.0'
    '1.2.3'
    em lista de inteiros [1], [1,2,3]
    """
    if pd.isna(value):
        return []

    s = str(value).strip()

    # unifica separadores
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1, 2, 3]:
                phases.append(ph)
        except ValueError:
            pass

    return sorted(set(phases))

# Branches (MT)
df_branches["phase_list"] = df_branches["phase"].apply(normalize_phases)

# Trafos
df_trafos["phase_list"] = df_trafos["phases"].apply(normalize_phases)


In [ ]:
# Remover SourceBus
df_buses = df_buses[~df_buses["name"].astype(str).str.contains("Source", case=False)]

# Mapeamento name -> N (id numérico)
bus_id_map = dict(zip(df_buses["name"], df_buses["N"]))

# Lista final de barras MT
buses_mt = df_buses["N"].astype(int).unique().tolist()


In [ ]:
# Converter barras l,k para identificador numérico
df_branches["from_bus"] = df_branches["l"].map(bus_id_map)
df_branches["to_bus"]   = df_branches["k"].map(bus_id_map)

# Remover ramos mal mapeados
df_branches = df_branches.dropna(subset=["from_bus", "to_bus"])

df_branches["from_bus"] = df_branches["from_bus"].astype(int)
df_branches["to_bus"]   = df_branches["to_bus"].astype(int)
df_branches["m"]        = df_branches["m"].astype(int)

# Conjunto de ramos MT
branches_mt = df_branches["m"].unique().tolist()


In [ ]:
PHASES = [1, 2, 3]

branch_phase = []

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # garante inteiro

    raw = row["phase"]

    if pd.isna(raw):
        continue

    s = str(raw).strip()

    # Normalização:
    # troca qualquer separador por vírgula
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    for p in parts:
        # extrai apenas números válidos
        try:
            ph = int(float(p))
        except ValueError:
            continue

        if ph in PHASES:
            branch_phase.append((m, ph))

branch_phase = sorted(set(branch_phase))


In [ ]:
R = {}
X = {}
Imax = {}
from_bus = {}
to_bus = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # robusto se vier 1.0

    from_bus[m] = int(row["from_bus"])
    to_bus[m]   = int(row["to_bus"])
    Imax[m]     = float(row["Imax"])

    # USA A COLUNA NORMALIZADA
    for ph in row["phase_list"]:
        R[(m, ph)] = float(row["R"])
        X[(m, ph)] = float(row["X"])



Length = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    Length[m] = float(row["Length_m"])

Vbase = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    fb = from_bus[m]
    Vbase[m] = float(df_buses.loc[df_buses["N"] == fb, "v_nom_kv"].iloc[0])
Smax = {}

for m in branches_mt:
    Smax[m] = Vbase[m] * Imax[m]  # kVA por fase


In [ ]:
# Padronizar mv_bus
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].map(bus_id_map)
df_trafos = df_trafos.dropna(subset=["mv_bus_num"])
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)

# Placeholder de potência vinda da BT
P_TR = {}
Q_TR = {}

for _, row in df_trafos.iterrows():
    mvb = int(row["mv_bus_num"])

    # USA DIRETAMENTE A LISTA NORMALIZADA
    for ph in row["phase_list"]:
        P_TR[(mvb, ph)] = 0.0
        Q_TR[(mvb, ph)] = 0.0


In [ ]:
trafo_mv_bus = {
    row["trafo_id"]: int(row["mv_bus_num"])
    for _, row in df_trafos.iterrows()
}


alpha_tr = {}
beta_tr  = {}

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    r_per = float(row["r_per"]) / 100.0
    xhl   = float(row["XHL"]) / 100.0

    for ph in row["phase_list"]:
        alpha_tr[(tid, ph)] = r_per
        beta_tr[(tid, ph)]  = xhl


In [ ]:
model = ConcreteModel()

# Conjuntos
model.BUS = Set(initialize=buses_mt)
T = list(range(0, 2881))  # 0 ... 2880

model.T = Set(initialize=T, ordered=True)


bus_phase = set()

for (br, ph) in branch_phase:
    bus_phase.add((from_bus[br], ph))
    bus_phase.add((to_bus[br], ph))

model.BUS_PH = Set(dimen=2, initialize=bus_phase)


model.PH  = Set(initialize=PHASES)
model.BR  = Set(initialize=branches_mt)
model.BR_PH = Set(initialize=branch_phase, dimen=2)

trafo_phase = []

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    for ph in row["phase_list"]:
        trafo_phase.append((tid, ph))

model.TR_PH = Set(dimen=2, initialize=trafo_phase)


# Parâmetros
model.R = Param(model.BR_PH, initialize=R)
model.X = Param(model.BR_PH, initialize=X)

model.from_bus = Param(model.BR, initialize=from_bus)
model.to_bus   = Param(model.BR, initialize=to_bus)



# Variáveis
model.P = Var(model.BR_PH)
model.Q = Var(model.BR_PH)
model.v = Var(model.BUS, model.PH, bounds=(0.95**2, 1.05**2))

model.Ppos = Var(model.BR_PH, domain=NonNegativeReals)
model.Pneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Qpos = Var(model.BR_PH, domain=NonNegativeReals)
model.Qneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Ploss = Var(model.BR_PH, domain=NonNegativeReals)




In [ ]:
# Potência aparente "equivalente" (L1) por trafo e fase
model.Str_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Str_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Ploss_tr = Var(model.TR_PH, domain=NonNegativeReals)
model.Qloss_tr = Var(model.TR_PH, domain=NonNegativeReals)

model.Ptr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Ptr_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_neg = Var(model.TR_PH, domain=NonNegativeReals)




In [ ]:
def Ptr_def(m, tid, ph):
    return m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]

def Qtr_def(m, tid, ph):
    return m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]


In [ ]:
def voltage_drop_rule(m, br, ph):
    i = m.from_bus[br]
    j = m.to_bus[br]
    return m.v[j,ph] == m.v[i,ph] - 2*(
        (m.R[br,ph]*Length[br]) * m.P[br,ph] +
        (m.X[br,ph]*Length[br]) * m.Q[br,ph]
    )



In [ ]:
def Ploss_rule(m, br, ph):
    return m.Ploss[br,ph] >= (
        m.R[br,ph] *
        (m.Ppos[br,ph] + m.Pneg[br,ph]
         + m.Qpos[br,ph] + m.Qneg[br,ph])
    )

model.PlossDef = Constraint(model.BR_PH, rule=Ploss_rule)


In [ ]:
def active_balance_rule(m, bus, ph):

    inflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph] + m.Ploss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

def reactive_balance_rule(m, bus, ph):

    inflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph] + m.Qloss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

model.PBalance = Constraint(model.BUS_PH, rule=active_balance_rule)
model.QBalance = Constraint(model.BUS_PH, rule=reactive_balance_rule)


In [ ]:
Vmin = 0.95  # pu

def current_limit_rule(m, br, ph):
    return (
        m.Ppos[br,ph] + m.Pneg[br,ph]
        + m.Qpos[br,ph] + m.Qneg[br,ph]
        <= Smax[br] * Vmin
    )

model.CurrentLimit = Constraint(model.BR_PH, rule=current_limit_rule)



In [ ]:
slack_bus = int(df_buses[df_buses["tb"] == 3]["N"].iloc[0])

for ph in PHASES:
    model.v[slack_bus, ph].fix(1.0)


In [ ]:
def Ploss_tr_rule(m, tid, ph):
    return m.Ploss_tr[tid,ph] >= alpha_tr[(tid,ph)] * (
        m.Ptr_pos[tid,ph] + m.Ptr_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.PlossTrDef = Constraint(model.TR_PH, rule=Ploss_tr_rule)


def Qloss_tr_rule(m, tid,ph):
    return m.Qloss_tr[tid,ph] >= beta_tr[(tid,ph)] * (
        m.Str_pos[tid,ph] + m.Str_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.QlossTrDef = Constraint(model.TR_PH, rule=Qloss_tr_rule)



# BT

In [ ]:
# --- Ler BT e PV (mesmo branches_1.xlsx / buses_1.xlsx) ---
df_branches_bt = pd.read_excel(branches_path, sheet_name="BT")
df_buses_bt    = pd.read_excel(buses_path,    sheet_name="BT")
df_pv          = pd.read_excel(buses_path,    sheet_name="PV")  # PV está em buses_1.xlsx

# Normalizar fases em BT e PV
df_branches_bt["phase_list"] = df_branches_bt["phase"].apply(normalize_phases)
df_buses_bt["phase_list"]    = df_buses_bt["phases"].apply(normalize_phases)
df_pv["phase_list"]          = df_pv["phases"].apply(normalize_phases)

# Map name -> N na BT
bus_id_map_bt = dict(zip(df_buses_bt["name"], df_buses_bt["N"]))

buses_bt = df_buses_bt["N"].astype(int).unique().tolist()


In [ ]:
# Mapear ramos BT: l,k (nomes) -> N
df_branches_bt["from_bus"] = df_branches_bt["l"].map(bus_id_map_bt)
df_branches_bt["to_bus"]   = df_branches_bt["k"].map(bus_id_map_bt)
df_branches_bt = df_branches_bt.dropna(subset=["from_bus", "to_bus"])

df_branches_bt["from_bus"] = df_branches_bt["from_bus"].astype(int)
df_branches_bt["to_bus"]   = df_branches_bt["to_bus"].astype(int)
df_branches_bt["m"]        = df_branches_bt["m"].astype(int)

branches_bt = df_branches_bt["m"].unique().tolist()

# Ramos por fase (BT)
branch_phase_bt = []
for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    for ph in row["phase_list"]:
        branch_phase_bt.append((m, ph))
branch_phase_bt = sorted(set(branch_phase_bt))

# Barras por fase (BT)
bus_phase_bt = set()
for (br, ph) in branch_phase_bt:
    fb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "from_bus"].iloc[0])
    tb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "to_bus"].iloc[0])
    bus_phase_bt.add((fb, ph))
    bus_phase_bt.add((tb, ph))
bus_phase_bt = sorted(bus_phase_bt)


In [ ]:
R_bt = {}
X_bt = {}
Imax_bt = {}
from_bus_bt = {}
to_bus_bt = {}
Length_bt_km = {}

for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    from_bus_bt[m] = int(row["from_bus"])
    to_bus_bt[m]   = int(row["to_bus"])
    Imax_bt[m]     = float(row["Imax"])

    Lkm = float(row["Length_m"]) / 1000.0
    Length_bt_km[m] = Lkm

    for ph in row["phase_list"]:
        # R,X em ohm/km -> total em ohm
        R_bt[(m, ph)] = float(row["R"]) * Lkm
        X_bt[(m, ph)] = float(row["X"]) * Lkm

# Vbase por ramo BT (kV da barra de origem)
Vbase_bt = {}
for m in branches_bt:
    fb = from_bus_bt[m]
    Vbase_bt[m] = float(df_buses_bt.loc[df_buses_bt["N"] == fb, "v_nom_kv"].iloc[0])

# Smax BT (kVA por fase) para limite linear de corrente
Smax_bt = {m: Vbase_bt[m] * Imax_bt[m] for m in branches_bt}


In [ ]:
Pload_bt = {}
Qload_bt = {}

for _, row in df_buses_bt.iterrows():
    b = int(row["N"])
    phs = row["phase_list"]
    if len(phs) == 0:
        continue

    Pd = float(row["P_D"]) if not pd.isna(row["P_D"]) else 0.0
    Qd = float(row["Q_D"]) if not pd.isna(row["Q_D"]) else 0.0

    share = 1.0 / len(phs)

    for ph in phs:
        Pload_bt[(b, ph)] = Pd * share
        Qload_bt[(b, ph)] = Qd * share


In [ ]:
# PV por barra-fase
Ppv_bt = {}     # param (injeção ativa)
Qpv_max = {}    # param (limite reativo)

# Inicializa com zero para todas as barras/fases (se não houver PV)
for (b, ph) in bus_phase_bt:
    Ppv_bt[(b, ph)] = 0.0
    Qpv_max[(b, ph)] = 0.0

for _, row in df_pv.iterrows():
    bus_name = row["Bus"]
    if bus_name not in bus_id_map_bt:
        continue
    b = int(bus_id_map_bt[bus_name])

    # para cada fase, lê colunas p_pv_x e q_pv_x
    for ph in row["phase_list"]:
        pcol = f"p_pv_{ph}"
        qcol = f"q_pv_{ph}"
        Ppv_bt[(b, ph)] = float(row[pcol]) if pcol in row and not pd.isna(row[pcol]) else 0.0
        Qpv_max[(b, ph)] = float(row[qcol]) if qcol in row and not pd.isna(row[qcol]) else 0.0


In [ ]:
# Mapear lv_bus (nome) -> N BT
df_trafos["lv_bus_num"] = df_trafos["lv_bus"].map(bus_id_map_bt)
df_trafos = df_trafos.dropna(subset=["lv_bus_num"])
df_trafos["lv_bus_num"] = df_trafos["lv_bus_num"].astype(int)

trafo_lv_bus = {row["trafo_id"]: int(row["lv_bus_num"]) for _, row in df_trafos.iterrows()}


In [ ]:
# Conjuntos BT
model.BUS_BT = Set(initialize=buses_bt)
model.BUS_PH_BT = Set(dimen=2, initialize=bus_phase_bt)

model.BR_BT = Set(initialize=branches_bt)
model.BR_PH_BT = Set(dimen=2, initialize=branch_phase_bt)

# Parâmetros BT
model.R_BT = Param(model.BR_PH_BT, initialize=R_bt)
model.X_BT = Param(model.BR_PH_BT, initialize=X_bt)

model.from_bus_bt = Param(model.BR_BT, initialize=from_bus_bt)
model.to_bus_bt   = Param(model.BR_BT, initialize=to_bus_bt)

model.Pload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Pload_bt.get((b,ph), 0.0), default=0.0)
model.Qload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qload_bt.get((b,ph), 0.0), default=0.0)

model.Ppv_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Ppv_bt.get((b,ph), 0.0), default=0.0)
model.Qpv_max = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qpv_max.get((b,ph), 0.0), default=0.0)


In [ ]:
# Variáveis BT
model.P_BT = Var(model.BR_PH_BT)
model.Q_BT = Var(model.BR_PH_BT)
model.v_BT = Var(model.BUS_BT, model.PH, bounds=(0.95**2, 1.05**2))

# Decomposição abs BT
model.Ppos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Pneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qpos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)

# PV: q variável por barra-fase (inicial 0), limitada por Qpv_max
def qpv_bounds(m, b, ph):
    qmax = value(m.Qpv_max[b,ph])
    return (-qmax, qmax)

model.Qpv = Var(model.BUS_BT, model.PH, bounds=qpv_bounds, initialize=0.0)

# PF como variável "rastreamento" (init 1). (acoplamento pf↔Q será feito depois via MILP/PWL)
model.pf_pv = Var(model.BUS_BT, model.PH, bounds=(0.0, 1.0), initialize=1.0)


In [ ]:
def vdrop_bt_rule(m, br, ph):
    i = m.from_bus_bt[br]
    j = m.to_bus_bt[br]
    return m.v_BT[j,ph] == m.v_BT[i,ph] - 2*(m.R_BT[br,ph]*m.P_BT[br,ph] + m.X_BT[br,ph]*m.Q_BT[br,ph])

model.VdropBT = Constraint(model.BR_PH_BT, rule=vdrop_bt_rule)


In [ ]:
def pbal_bt_rule(m, bus, ph):

    inflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    # Soma potência dos trafos cujo lv_bus é esta barra (pode haver mais de um)
    Ptr_here = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Pload_bt[bus,ph] - m.Ppv_bt[bus,ph] - Ptr_here

model.PBalanceBT = Constraint(model.BUS_PH_BT, rule=pbal_bt_rule)


def qbal_bt_rule(m, bus, ph):

    inflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    Qtr_here = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Qload_bt[bus,ph] - m.Qpv[bus,ph] - Qtr_here

model.QBalanceBT = Constraint(model.BUS_PH_BT, rule=qbal_bt_rule)


In [ ]:
def Psplit_bt_rule(m, br, ph):
    return m.P_BT[br,ph] == m.Ppos_BT[br,ph] - m.Pneg_BT[br,ph]
def Qsplit_bt_rule(m, br, ph):
    return m.Q_BT[br,ph] == m.Qpos_BT[br,ph] - m.Qneg_BT[br,ph]

model.PsplitBT = Constraint(model.BR_PH_BT, rule=Psplit_bt_rule)
model.QsplitBT = Constraint(model.BR_PH_BT, rule=Qsplit_bt_rule)

Vmin_bt = 0.95

def current_limit_bt_rule(m, br, ph):
    return (m.Ppos_BT[br,ph] + m.Pneg_BT[br,ph] + m.Qpos_BT[br,ph] + m.Qneg_BT[br,ph]) <= (Smax_bt[br] * Vmin_bt)

model.CurrentLimitBT = Constraint(model.BR_PH_BT, rule=current_limit_bt_rule)


In [ ]:
bt_slacks = df_buses_bt[df_buses_bt["tb"] == 3]["N"].astype(int).tolist()
if len(bt_slacks) > 0:
    bt_slack = bt_slacks[0]
    for ph in PHASES:
        model.v_BT[bt_slack, ph].fix(1.0)


# Regulador

In [ ]:
# Mapeamento explícito regulador → fase
reg_phase_map = {
    "Reg1": 1,
    "Reg2": 2,
    "Reg3": 3,
}

reg_phase = [(rid, ph) for rid, ph in reg_phase_map.items()]

model.REG_PH = Set(dimen=2, initialize=reg_phase)



This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
reg_mv_bus = {row["trafo_id"]: int(bus_id_map[row["mv_bus"]]) for _, row in df_reg.iterrows()}
reg_lv_bus = {row["trafo_id"]: int(bus_id_map[row["lv_bus"]]) for _, row in df_reg.iterrows()}

vreg = {}
band = {}
step = {}
tap_min = {}
tap_max = {}

NTAPS = 16  # <<< DEFINIÇÃO CORRETA (engenharia)

for _, row in df_reg.iterrows():
    rid = row["trafo_id"]
    if rid not in reg_phase_map:
        continue

    ph = reg_phase_map[rid]

    vreg[(rid,ph)] = row["vreg_V"] / (row["kv_mv"] * 1000)
    band[(rid,ph)] = row["band_V"] / (row["kv_mv"] * 1000)

    # Step (% → pu)
    step[(rid,ph)] = float(row["Step"]) / 100.0

    tap_min[(rid,ph)] = -NTAPS
    tap_max[(rid,ph)] =  NTAPS


In [ ]:
model.tap = Var(model.REG_PH, model.T, domain=Integers)


In [ ]:
def tap_bounds(m, rid, ph, t):
    return (tap_min[(rid,ph)], tap_max[(rid,ph)])

model.tap = Var(model.REG_PH, model.T, domain=Integers, bounds=tap_bounds)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# NÃO aplicar regulador se j == mv_bus do trafo
def reg_voltage(mm, rid, ph, t):
    i = reg_mv_bus[rid]
    j = reg_lv_bus[rid]
    if j in trafo_mv_bus.values():
        return Constraint.Skip
    return mm.v_MT[j,ph,t] == mm.v_MT[i,ph,t] + 2*step[(rid,ph)]*mm.tap[rid,ph,t]


model.RegVoltage = Constraint(model.REG_PH, model.T, rule=reg_voltage_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
model.over_v = Var(model.REG_PH, model.T, domain=Binary)
model.under_v = Var(model.REG_PH, model.T, domain=Binary)


In [ ]:
M = 10.0

def over_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return m.v[j,ph] - (vreg[(rid,ph)] + band[(rid,ph)]/2) <= M * m.over_v[rid,ph,t]

def under_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return (vreg[(rid,ph)] - band[(rid,ph)]/2) - m.v[j,ph] <= M * m.under_v[rid,ph,t]

model.OverV = Constraint(model.REG_PH, model.T, rule=over_voltage_rule)
model.UnderV = Constraint(model.REG_PH, model.T, rule=under_voltage_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
t0 = T_list[0]
for (rid,ph) in m.REG_PH:
    m.tap[rid,ph,t0].fix(0)
    m.tap_up[rid,ph,t0].fix(0)
    m.tap_down[rid,ph,t0].fix(0)



def tap_evolution_rule(m, rid, ph, t):
    if t == 0:
        return Constraint.Skip
    return (
        m.tap[rid,ph,t]
        == m.tap[rid,ph,t-1]
        + m.tap_up[rid,ph,t]
        - m.tap_down[rid,ph,t]
    )

model.TapEvolution = Constraint(model.REG_PH, model.T, rule=tap_evolution_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
def tap_up_rule(m, rid, ph, t):
    return m.tap_up[rid,ph,t] <= m.under_v[rid,ph,t]

def tap_down_rule(m, rid, ph, t):
    return m.tap_down[rid,ph,t] <= m.over_v[rid,ph,t]

model.TapUpRule = Constraint(model.REG_PH, model.T, rule=tap_up_rule)
model.TapDownRule = Constraint(model.REG_PH, model.T, rule=tap_down_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
model.obj = Objective(
    expr=sum(
        model.tap_up[rid,ph,t] + model.tap_down[rid,ph,t]
        for (rid,ph) in model.REG_PH
        for t in model.T
    ),
    sense=minimize
)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
solver = SolverFactory("highs")
result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 51855 rows; 43221 cols; 129633 nonzeros; 43215 integer variables (34572 binary)
Coefficient ranges:
  Matrix  [1e-04, 1e+01]
  Cost    [1e+00, 1e+00]
  Bound   [9e-01, 2e+01]
  RHS     [5e-02, 5e-02]
Presolving model
25923 rows, 25929 cols, 69129 nonzeros  0s
0 rows, 8643 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-51855); columns 0(-43221); nonzeros 0(-129633) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  D

# V2

In [267]:
!pip -q install pyomo pandas openpyxl highspy



In [285]:
# base_path = "/content/ACOPF"
# ============================================================
# 0) CONSTANTES / CONTROLE
# ============================================================
PHASES = [1,2,3]

SOLAR_START = 196
SOLAR_END   = 387
T_LIST = list(range(SOLAR_START, SOLAR_END+1))   # reduz esforço computacional

# Tensões (pu) -> modelo usa pu²
v_min_pu = 0.93
v_max_pu = 1.05
V2_MIN = v_min_pu**2
V2_MAX = v_max_pu**2


# FP mínimo PV
pf_min = 0.90
k_pf   = math.tan(math.acos(pf_min))  # |Q| <= k_pf * P

# Tap step (0.625% = 0.00625)
delta_v = 0.00625

# Faixa de ratio observada no OpenDSS (ex.: [1.000, 1.031])
tap_ratio_min = 1.000
tap_ratio_max = 1.031

# Converte para taps (inteiro)
NTAPS = int(round((tap_ratio_max - 1.0) / delta_v))  # ~5
# Nós consideraremos taps de 0..NTAPS (ratio>=1). Se quiser simétrico, mude para -NTAPS..+NTAPS.
TAP_SET = list(range(0, NTAPS+1))  # discreto: 0..NTAPS
TAP_RATIO = {k: (1.0 + delta_v*k) for k in TAP_SET}
TAP_RATIO2 = {k: (TAP_RATIO[k]**2) for k in TAP_SET}

# Big-M para McCormick (como v em pu² está ~[0.86..1.1025], M não precisa ser gigante)
# Usaremos bounds explícitos, então não é necessário Big-M aqui; McCormick usa bounds.
M_V2_MIN = V2_MIN
M_V2_MAX = V2_MAX

In [286]:
def normalize_phases(value):
    """Aceita 1, 1.0, '1,2,3', '1.2.3' e devolve lista [1,2,3]."""
    if pd.isna(value):
        return []
    s = str(value).strip().replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip()!=""]
    out=[]
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1,2,3]:
                out.append(ph)
        except:
            pass
    return sorted(set(out))


def build_prev_map(T):
    T_sorted = sorted(T)
    prev = {T_sorted[i]: T_sorted[i-1] for i in range(1, len(T_sorted))}
    return T_sorted[0], prev



In [287]:
# ============================================================
# 2) LOAD DATA (GitHub)
# ============================================================
# ajuste base_path se necessário
base_path = "/content/ACOPF"
!rm -rf /content/ACOPF
!git clone -q https://github.com/DarleneJD/ACOPF.git

branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

# ---------- MT ----------
df_branches_mt = pd.read_excel(branches_path, sheet_name="MT")
df_trafos      = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses_mt    = pd.read_excel(buses_path,    sheet_name="MT")

df_branches_mt["phase_list"] = df_branches_mt["phase"].apply(normalize_phases)
df_trafos["phase_list"]      = df_trafos["phases"].apply(normalize_phases)

# remove SourceBus (AT)
df_buses_mt = df_buses_mt[~df_buses_mt["name"].astype(str).str.contains("Source", case=False)].copy()
bus_id_map_mt = dict(zip(df_buses_mt["name"].astype(str), df_buses_mt["N"].astype(int)))

# >>> REMOVE o ramo MT 650->RG60 (é SVR no OpenDSS, não linha)
df_branches_mt = df_branches_mt[
    ~((df_branches_mt["l"].astype(str)=="650") & (df_branches_mt["k"].astype(str)=="RG60"))
].copy()

# mapeia buses
df_branches_mt["from_bus"] = df_branches_mt["l"].astype(str).map(bus_id_map_mt)
df_branches_mt["to_bus"]   = df_branches_mt["k"].astype(str).map(bus_id_map_mt)
df_branches_mt = df_branches_mt.dropna(subset=["from_bus","to_bus"]).copy()
df_branches_mt["from_bus"] = df_branches_mt["from_bus"].astype(int)
df_branches_mt["to_bus"]   = df_branches_mt["to_bus"].astype(int)
df_branches_mt["m"]        = df_branches_mt["m"].astype(int)

buses_mt    = sorted(df_buses_mt["N"].astype(int).unique().tolist())
branches_mt = sorted(df_branches_mt["m"].astype(int).unique().tolist())

BR_PH_MT = sorted({(int(r["m"]), ph) for _,r in df_branches_mt.iterrows() for ph in r["phase_list"]})
from_bus_mt = {int(r["m"]): int(r["from_bus"]) for _,r in df_branches_mt.iterrows()}
to_bus_mt   = {int(r["m"]): int(r["to_bus"])   for _,r in df_branches_mt.iterrows()}
Imax_mt     = {int(r["m"]): float(r["Imax"])   for _,r in df_branches_mt.iterrows()}

BUS_PH_MT = sorted({(from_bus_mt[br],ph) for (br,ph) in BR_PH_MT} | {(to_bus_mt[br],ph) for (br,ph) in BR_PH_MT})

# R,X MT (assumindo TOTAL por ramo, como você vinha usando)
R_mt = {(int(r["m"]),ph): float(r["R"]) for _,r in df_branches_mt.iterrows() for ph in r["phase_list"]}
X_mt = {(int(r["m"]),ph): float(r["X"]) for _,r in df_branches_mt.iterrows() for ph in r["phase_list"]}

In [288]:
# Smax MT (kVA/fase) envelope linear de corrente
Vmin_env = 0.95
Smax_mt = {}
for br in branches_mt:
    fb = from_bus_mt[br]
    Vbase_kv = float(df_buses_mt.loc[df_buses_mt["N"]==fb, "v_nom_kv"].iloc[0])
    Smax_mt[br] = Vbase_kv * Imax_mt[br]

In [289]:
# slack MT
slack_bus_mt = int(df_buses_mt.loc[df_buses_mt["tb"]==3, "N"].iloc[0])

In [290]:
# ---------- BT + PV ----------
df_branches_bt = pd.read_excel(branches_path, sheet_name="BT")
df_buses_bt    = pd.read_excel(buses_path,    sheet_name="BT")
df_pv_sheet    = pd.read_excel(buses_path,    sheet_name="PV")

df_branches_bt["phase_list"] = df_branches_bt["phase"].apply(normalize_phases)
df_buses_bt["phase_list"]    = df_buses_bt["phases"].apply(normalize_phases)
df_pv_sheet["phase_list"]    = df_pv_sheet["phases"].apply(normalize_phases)

bus_id_map_bt = dict(zip(df_buses_bt["name"].astype(str), df_buses_bt["N"].astype(int)))

df_branches_bt["from_bus"] = df_branches_bt["l"].astype(str).map(bus_id_map_bt)
df_branches_bt["to_bus"]   = df_branches_bt["k"].astype(str).map(bus_id_map_bt)
df_branches_bt = df_branches_bt.dropna(subset=["from_bus","to_bus"]).copy()
df_branches_bt["from_bus"] = df_branches_bt["from_bus"].astype(int)
df_branches_bt["to_bus"]   = df_branches_bt["to_bus"].astype(int)
df_branches_bt["m"]        = df_branches_bt["m"].astype(int)

buses_bt    = sorted(df_buses_bt["N"].astype(int).unique().tolist())
branches_bt = sorted(df_branches_bt["m"].astype(int).unique().tolist())

BR_PH_BT = sorted({(int(r["m"]), ph) for _,r in df_branches_bt.iterrows() for ph in r["phase_list"]})
from_bus_bt = {int(r["m"]): int(r["from_bus"]) for _,r in df_branches_bt.iterrows()}
to_bus_bt   = {int(r["m"]): int(r["to_bus"])   for _,r in df_branches_bt.iterrows()}
Imax_bt     = {int(r["m"]): float(r["Imax"])   for _,r in df_branches_bt.iterrows()}
Lkm_bt      = {int(r["m"]): float(r["Length_m"])/1000.0 for _,r in df_branches_bt.iterrows()}

BUS_PH_BT = sorted({(from_bus_bt[br],ph) for (br,ph) in BR_PH_BT} | {(to_bus_bt[br],ph) for (br,ph) in BR_PH_BT})

# R,X BT em ohm/km * km
R_bt = {(int(r["m"]),ph): float(r["R"])*Lkm_bt[int(r["m"])] for _,r in df_branches_bt.iterrows() for ph in r["phase_list"]}
X_bt = {(int(r["m"]),ph): float(r["X"])*Lkm_bt[int(r["m"])] for _,r in df_branches_bt.iterrows() for ph in r["phase_list"]}

In [291]:
# Smax BT (kVA/fase)
Smax_bt = {}
for br in branches_bt:
    fb = from_bus_bt[br]
    Vbase_kv = float(df_buses_bt.loc[df_buses_bt["N"]==fb, "v_nom_kv"].iloc[0])
    Smax_bt[br] = Vbase_kv * Imax_bt[br]

In [292]:
# Loads BT (constantes no tempo aqui)
Pload_bt = {}
Qload_bt = {}
for _,r in df_buses_bt.iterrows():
    b = int(r["N"])
    phs = normalize_phases(r["phases"])
    if len(phs)==0:
        continue
    Pd = float(r["P_D"]) if not pd.isna(r["P_D"]) else 0.0
    Qd = float(r["Q_D"]) if not pd.isna(r["Q_D"]) else 0.0
    share = 1.0/len(phs)
    for ph in phs:
        Pload_bt[(b,ph)] = Pd*share
        Qload_bt[(b,ph)] = Qd*share

In [293]:
# PV limites por barra-fase
Ppv_max = {(b,ph):0.0 for (b,ph) in BUS_PH_BT}
Qpv_max = {(b,ph):0.0 for (b,ph) in BUS_PH_BT}
Spv_max = {(b,ph):0.0 for (b,ph) in BUS_PH_BT}

for _,r in df_pv_sheet.iterrows():
    busname = str(r["Bus"])
    if busname not in bus_id_map_bt:
        continue
    b = int(bus_id_map_bt[busname])
    kva_total = float(r["kva"]) if not pd.isna(r["kva"]) else 0.0
    phs = r["phase_list"]
    nph = max(len(phs),1)
    s_per_phase = kva_total/nph
    for ph in phs:
        pcol = f"p_pv_{ph}"
        qcol = f"q_pv_{ph}"
        pmax = float(r[pcol]) if (pcol in r and not pd.isna(r[pcol])) else 0.0
        qmax = float(r[qcol]) if (qcol in r and not pd.isna(r[qcol])) else 0.0
        Ppv_max[(b,ph)] = pmax
        Qpv_max[(b,ph)] = abs(qmax)
        Spv_max[(b,ph)] = s_per_phase


In [294]:
# ---------- Trafos MT<->BT ----------
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].astype(str).map(bus_id_map_mt)
df_trafos["lv_bus_num"] = df_trafos["lv_bus"].astype(str).map(bus_id_map_bt)
df_trafos = df_trafos.dropna(subset=["mv_bus_num","lv_bus_num"]).copy()
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)
df_trafos["lv_bus_num"] = df_trafos["lv_bus_num"].astype(int)

TR_PH = sorted({(str(r["trafo_id"]),ph) for _,r in df_trafos.iterrows() for ph in r["phase_list"]})
trafo_mv_bus = {str(r["trafo_id"]): int(r["mv_bus_num"]) for _,r in df_trafos.iterrows()}
trafo_lv_bus = {str(r["trafo_id"]): int(r["lv_bus_num"]) for _,r in df_trafos.iterrows()}

alpha_tr = {}
beta_tr  = {}
for _,r in df_trafos.iterrows():
    tid = str(r["trafo_id"])
    r_per = float(r["r_per"])/100.0 if not pd.isna(r["r_per"]) else 0.0
    xhl   = float(r["XHL"]) /100.0 if not pd.isna(r["XHL"]) else 0.0
    for ph in r["phase_list"]:
        alpha_tr[(tid,ph)] = r_per
        beta_tr[(tid,ph)]  = xhl

In [295]:
# ---------- Reguladores (SVR) ----------
df_reg = pd.read_excel(branches_path, sheet_name="Reg")
reg_phase_map = {"Reg1":1, "Reg2":2, "Reg3":3}
REG_PH = sorted([(rid,ph) for rid,ph in reg_phase_map.items()])

reg_mv_bus = {str(r["trafo_id"]): int(bus_id_map_mt[str(r["mv_bus"])]) for _,r in df_reg.iterrows() if str(r["trafo_id"]) in reg_phase_map}
reg_lv_bus = {str(r["trafo_id"]): int(bus_id_map_mt[str(r["lv_bus"])]) for _,r in df_reg.iterrows() if str(r["trafo_id"]) in reg_phase_map}

# Potência nominal por fase do regulador
Sreg_max = {}
for _,r in df_reg.iterrows():
    rid = str(r["trafo_id"])
    if rid not in reg_phase_map:
        continue
    ph = reg_phase_map[rid]
    kva = float(r["kva"]) if not pd.isna(r["kva"]) else 0.0
    Sreg_max[(rid,ph)] = kva

# Impedância série do regulador (envelope estável). Se quiser usar XHL=0.01 do OpenDSS, mantenha pequeno,
# mas cuidado com escalas (P/Q em kW vs pu).
R_reg = {(rid,ph): 0.00 for (rid,ph) in REG_PH}
X_reg = {(rid,ph): 0.00 for (rid,ph) in REG_PH}



In [296]:
# ============================================================
# 3) PYOMO MODEL
# ============================================================
m = pyo.ConcreteModel()

In [297]:
# ----------------------------
# Sets
# ----------------------------
m.T  = pyo.Set(initialize=T_LIST, ordered=True)
m.PH = pyo.Set(initialize=PHASES)

m.BUS_MT    = pyo.Set(initialize=buses_mt)
m.BUS_PH_MT = pyo.Set(dimen=2, initialize=BUS_PH_MT)
m.BR_MT     = pyo.Set(initialize=branches_mt)
m.BR_PH_MT  = pyo.Set(dimen=2, initialize=BR_PH_MT)

m.BUS_BT    = pyo.Set(initialize=buses_bt)
m.BUS_PH_BT = pyo.Set(dimen=2, initialize=BUS_PH_BT)
m.BR_BT     = pyo.Set(initialize=branches_bt)
m.BR_PH_BT  = pyo.Set(dimen=2, initialize=BR_PH_BT)

m.TR_PH   = pyo.Set(dimen=2, initialize=TR_PH)
m.REG_PH  = pyo.Set(dimen=2, initialize=REG_PH)

m.TAP_K = pyo.Set(initialize=TAP_SET)  # posições discretas

In [298]:
# ----------------------------
# Params
# ----------------------------
m.R_MT  = pyo.Param(m.BR_PH_MT, initialize=R_mt)
m.X_MT  = pyo.Param(m.BR_PH_MT, initialize=X_mt)
m.frmMT = pyo.Param(m.BR_MT, initialize=from_bus_mt)
m.toMT  = pyo.Param(m.BR_MT, initialize=to_bus_mt)

m.R_BT  = pyo.Param(m.BR_PH_BT, initialize=R_bt)
m.X_BT  = pyo.Param(m.BR_PH_BT, initialize=X_bt)
m.frmBT = pyo.Param(m.BR_BT, initialize=from_bus_bt)
m.toBT  = pyo.Param(m.BR_BT, initialize=to_bus_bt)

m.Pload_bt = pyo.Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Pload_bt.get((b,ph),0.0), default=0.0)
m.Qload_bt = pyo.Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Qload_bt.get((b,ph),0.0), default=0.0)

m.Ppv_max = pyo.Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Ppv_max.get((b,ph),0.0), default=0.0)
m.Qpv_max = pyo.Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Qpv_max.get((b,ph),0.0), default=0.0)
m.Spv_max = pyo.Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Spv_max.get((b,ph),0.0), default=0.0)

In [299]:
# ----------------------------
# Variables - Fonte (subestação) no slack
# ----------------------------
m.Psub = pyo.Var(m.PH, m.T, domain=pyo.NonNegativeReals)
m.Qsub = pyo.Var(m.PH, m.T, domain=pyo.Reals)

In [300]:
# ----------------------------
# Variables - MT
# ----------------------------
m.P_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.Reals)
m.Q_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.Reals)
m.v_MT = pyo.Var(m.BUS_MT, m.PH, m.T, bounds=(V2_MIN, V2_MAX))

m.Ppos_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.NonNegativeReals)
m.Pneg_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.NonNegativeReals)
m.Qpos_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.NonNegativeReals)
m.Qneg_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.NonNegativeReals)

m.Ploss_MT = pyo.Var(m.BR_PH_MT, m.T, domain=pyo.NonNegativeReals)


In [301]:
# ----------------------------
# Variables - BT
# ----------------------------
m.P_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.Reals)
m.Q_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.Reals)
m.v_BT = pyo.Var(m.BUS_BT, m.PH, m.T, bounds=(V2_MIN, V2_MAX))

m.Ppos_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.NonNegativeReals)
m.Pneg_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.NonNegativeReals)
m.Qpos_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.NonNegativeReals)
m.Qneg_BT = pyo.Var(m.BR_PH_BT, m.T, domain=pyo.NonNegativeReals)


In [302]:
# PV (por barra-fase-tempo)
m.Ppv = pyo.Var(m.BUS_BT, m.PH, m.T, domain=pyo.NonNegativeReals)
m.Qpv = pyo.Var(m.BUS_BT, m.PH, m.T, domain=pyo.Reals)
m.Qpv_pos = pyo.Var(m.BUS_BT, m.PH, m.T, domain=pyo.NonNegativeReals)
m.Qpv_neg = pyo.Var(m.BUS_BT, m.PH, m.T, domain=pyo.NonNegativeReals)

In [303]:
# ----------------------------
# Variables - Trafos MT<->BT (acoplamento potência)
# ----------------------------
m.Ptr_pos = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)
m.Ptr_neg = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)
m.Qtr_pos = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)
m.Qtr_neg = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)

m.Ploss_tr = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)
m.Qloss_tr = pyo.Var(m.TR_PH, m.T, domain=pyo.NonNegativeReals)

In [304]:
# ----------------------------
# Variables - Regulador (SVR como elemento)
# ----------------------------
m.Preg = pyo.Var(m.REG_PH, m.T, domain=pyo.Reals)
m.Qreg = pyo.Var(m.REG_PH, m.T, domain=pyo.Reals)

m.Preg_pos = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)
m.Preg_neg = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)
m.Qreg_pos = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)
m.Qreg_neg = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)

m.Ploss_reg = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)
m.Qloss_reg = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)

In [305]:
# Tap seleção discreta
# y[k]=1 escolhe o tap k (k in TAP_SET)
m.ytap = pyo.Var(m.REG_PH, m.T, m.TAP_K, domain=pyo.Binary)

# wtap[k] = v_i * y[k]  (linearização McCormick)
m.wtap = pyo.Var(m.REG_PH, m.T, m.TAP_K, bounds=(0.0, V2_MAX))

# over/under e contagem de operação (opcional)
m.over_v  = pyo.Var(m.REG_PH, m.T, domain=pyo.Binary)
m.under_v = pyo.Var(m.REG_PH, m.T, domain=pyo.Binary)
m.tap_up  = pyo.Var(m.REG_PH, m.T, domain=pyo.Binary)
m.tap_dn  = pyo.Var(m.REG_PH, m.T, domain=pyo.Binary)

In [306]:
# ============================================================
# 4) RESTRIÇÕES (SEPARADAS POR BLOCO)
# ============================================================

# ------------------------------------------------------------
# (A) Split P/Q e limites de corrente (MT)
# ------------------------------------------------------------
def mt_split_P(mm, br, ph, t):
    return mm.P_MT[br,ph,t] == mm.Ppos_MT[br,ph,t] - mm.Pneg_MT[br,ph,t]
def mt_split_Q(mm, br, ph, t):
    return mm.Q_MT[br,ph,t] == mm.Qpos_MT[br,ph,t] - mm.Qneg_MT[br,ph,t]
m.MT_Psplit = pyo.Constraint(m.BR_PH_MT, m.T, rule=mt_split_P)
m.MT_Qsplit = pyo.Constraint(m.BR_PH_MT, m.T, rule=mt_split_Q)

def mt_curr_limit(mm, br, ph, t):
    return (mm.Ppos_MT[br,ph,t] + mm.Pneg_MT[br,ph,t] + mm.Qpos_MT[br,ph,t] + mm.Qneg_MT[br,ph,t]) <= Smax_mt[br]*Vmin_env
m.MT_Ilim = pyo.Constraint(m.BR_PH_MT, m.T, rule=mt_curr_limit)

def mt_ploss_env(mm, br, ph, t):
    return mm.Ploss_MT[br,ph,t] >= mm.R_MT[br,ph]*(mm.Ppos_MT[br,ph,t]+mm.Pneg_MT[br,ph,t]+mm.Qpos_MT[br,ph,t]+mm.Qneg_MT[br,ph,t])
m.MT_PlossEnv = pyo.Constraint(m.BR_PH_MT, m.T, rule=mt_ploss_env)


In [307]:
# ------------------------------------------------------------
# (B) Queda de tensão LinDistFlow (MT)
# ------------------------------------------------------------
def mt_vdrop(mm, br, ph, t):
    i = mm.frmMT[br]; j = mm.toMT[br]
    return mm.v_MT[j,ph,t] == mm.v_MT[i,ph,t] - 2*(mm.R_MT[br,ph]*mm.P_MT[br,ph,t] + mm.X_MT[br,ph]*mm.Q_MT[br,ph,t])
m.MT_Vdrop = pyo.Constraint(m.BR_PH_MT, m.T, rule=mt_vdrop)


In [308]:
# ------------------------------------------------------------
# (C) Split P/Q e limites de corrente (BT)
# ------------------------------------------------------------
def bt_split_P(mm, br, ph, t):
    return mm.P_BT[br,ph,t] == mm.Ppos_BT[br,ph,t] - mm.Pneg_BT[br,ph,t]
def bt_split_Q(mm, br, ph, t):
    return mm.Q_BT[br,ph,t] == mm.Qpos_BT[br,ph,t] - mm.Qneg_BT[br,ph,t]
m.BT_Psplit = pyo.Constraint(m.BR_PH_BT, m.T, rule=bt_split_P)
m.BT_Qsplit = pyo.Constraint(m.BR_PH_BT, m.T, rule=bt_split_Q)

def bt_curr_limit(mm, br, ph, t):
    return (mm.Ppos_BT[br,ph,t] + mm.Pneg_BT[br,ph,t] + mm.Qpos_BT[br,ph,t] + mm.Qneg_BT[br,ph,t]) <= Smax_bt[br]*Vmin_env
m.BT_Ilim = pyo.Constraint(m.BR_PH_BT, m.T, rule=bt_curr_limit)

In [309]:
# ------------------------------------------------------------
# (D) Queda de tensão LinDistFlow (BT)
# ------------------------------------------------------------
def bt_vdrop(mm, br, ph, t):
    i = mm.frmBT[br]; j = mm.toBT[br]
    return mm.v_BT[j,ph,t] == mm.v_BT[i,ph,t] - 2*(mm.R_BT[br,ph]*mm.P_BT[br,ph,t] + mm.X_BT[br,ph]*mm.Q_BT[br,ph,t])
m.BT_Vdrop = pyo.Constraint(m.BR_PH_BT, m.T, rule=bt_vdrop)

In [310]:
# ------------------------------------------------------------
# (E) Transformadores MT<->BT (perdas e acoplamento de tensão)
# ------------------------------------------------------------
def tr_ploss(mm, tid, ph, t):
    a = alpha_tr.get((tid,ph),0.0)
    return mm.Ploss_tr[tid,ph,t] >= a*(mm.Ptr_pos[tid,ph,t]+mm.Ptr_neg[tid,ph,t]+mm.Qtr_pos[tid,ph,t]+mm.Qtr_neg[tid,ph,t])
m.TR_Ploss = pyo.Constraint(m.TR_PH, m.T, rule=tr_ploss)

def tr_qloss(mm, tid, ph, t):
    b = beta_tr.get((tid,ph),0.0)
    return mm.Qloss_tr[tid,ph,t] >= b*(mm.Ptr_pos[tid,ph,t]+mm.Ptr_neg[tid,ph,t]+mm.Qtr_pos[tid,ph,t]+mm.Qtr_neg[tid,ph,t])
m.TR_Qloss = pyo.Constraint(m.TR_PH, m.T, rule=tr_qloss)

def tr_vcouple(mm, tid, ph, t):
    mvb = trafo_mv_bus[tid]
    lvb = trafo_lv_bus[tid]
    return mm.v_BT[lvb,ph,t] == mm.v_MT[mvb,ph,t]
m.TR_Vcouple = pyo.Constraint(m.TR_PH, m.T, rule=tr_vcouple)

In [311]:
# ------------------------------------------------------------
# (F) PV (BT) — limites + capability + fp mínimo
# ------------------------------------------------------------
def qpv_split(mm, b, ph, t):
    return mm.Qpv[b,ph,t] == mm.Qpv_pos[b,ph,t] - mm.Qpv_neg[b,ph,t]
m.PV_Qsplit = pyo.Constraint(m.BUS_BT, m.PH, m.T, rule=qpv_split)

def pv_Pmax(mm, b, ph, t):
    return mm.Ppv[b,ph,t] <= mm.Ppv_max[b,ph]
m.PV_Pmax = pyo.Constraint(m.BUS_BT, m.PH, m.T, rule=pv_Pmax)

def pv_Qmax(mm, b, ph, t):
    return mm.Qpv_pos[b,ph,t] + mm.Qpv_neg[b,ph,t] <= mm.Qpv_max[b,ph]
m.PV_Qmax = pyo.Constraint(m.BUS_BT, m.PH, m.T, rule=pv_Qmax)

def pv_Scap(mm, b, ph, t):
    # envelope linear (L1). Se quiser mais fiel, dá pra usar PWL.
    return mm.Ppv[b,ph,t] + mm.Qpv_pos[b,ph,t] + mm.Qpv_neg[b,ph,t] <= mm.Spv_max[b,ph]
m.PV_Scap = pyo.Constraint(m.BUS_BT, m.PH, m.T, rule=pv_Scap)

def pv_fp(mm, b, ph, t):
    # |Q| <= k_pf * P  (linear)
    return mm.Qpv_pos[b,ph,t] + mm.Qpv_neg[b,ph,t] <= k_pf * mm.Ppv[b,ph,t]
m.PV_FP = pyo.Constraint(m.BUS_BT, m.PH, m.T, rule=pv_fp)


In [312]:
# ------------------------------------------------------------
# (G) Balanços nodais (BT) com PV e trafos
# ------------------------------------------------------------
def pbal_bt(mm, bus, ph, t):
    inflow  = sum(mm.P_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.toBT[br]==bus)
    outflow = sum(mm.P_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.frmBT[br]==bus)

    tr_inj = sum((mm.Ptr_pos[tid,ph,t]-mm.Ptr_neg[tid,ph,t])
                 for (tid,p) in mm.TR_PH if p==ph and trafo_lv_bus[tid]==bus)

    return inflow - outflow == mm.Pload_bt[bus,ph] - mm.Ppv[bus,ph,t] - tr_inj
m.BT_Pbal = pyo.Constraint(m.BUS_PH_BT, m.T, rule=pbal_bt)

def qbal_bt(mm, bus, ph, t):
    inflow  = sum(mm.Q_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.toBT[br]==bus)
    outflow = sum(mm.Q_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.frmBT[br]==bus)

    tr_inj = sum((mm.Qtr_pos[tid,ph,t]-mm.Qtr_neg[tid,ph,t])
                 for (tid,p) in mm.TR_PH if p==ph and trafo_lv_bus[tid]==bus)

    return inflow - outflow == mm.Qload_bt[bus,ph] - mm.Qpv[bus,ph,t] - tr_inj
m.BT_Qbal = pyo.Constraint(m.BUS_PH_BT, m.T, rule=qbal_bt)

In [313]:
# ------------------------------------------------------------
# (H) Regulador SVR como elemento: split + perdas + limite S
# ------------------------------------------------------------
def reg_psplit(mm, rid, ph, t):
    return mm.Preg[rid,ph,t] == mm.Preg_pos[rid,ph,t] - mm.Preg_neg[rid,ph,t]
def reg_qsplit(mm, rid, ph, t):
    return mm.Qreg[rid,ph,t] == mm.Qreg_pos[rid,ph,t] - mm.Qreg_neg[rid,ph,t]
m.REG_Psplit = pyo.Constraint(m.REG_PH, m.T, rule=reg_psplit)
m.REG_Qsplit = pyo.Constraint(m.REG_PH, m.T, rule=reg_qsplit)

def reg_ploss(mm, rid, ph, t):
    # envelope L1 (se R_reg=0, zera as perdas)
    return mm.Ploss_reg[rid,ph,t] >= R_reg[(rid,ph)]*(mm.Preg_pos[rid,ph,t]+mm.Preg_neg[rid,ph,t]+mm.Qreg_pos[rid,ph,t]+mm.Qreg_neg[rid,ph,t])
m.REG_Ploss = pyo.Constraint(m.REG_PH, m.T, rule=reg_ploss)

def reg_slimit(mm, rid, ph, t):
    return (mm.Preg_pos[rid,ph,t]+mm.Preg_neg[rid,ph,t]+mm.Qreg_pos[rid,ph,t]+mm.Qreg_neg[rid,ph,t]) <= Sreg_max[(rid,ph)]
m.REG_Slimit = pyo.Constraint(m.REG_PH, m.T, rule=reg_slimit)

In [314]:
# ------------------------------------------------------------
# (I) SVR: seleção discreta tap + McCormick + equação de tensão
#     v_j = a(k)^2 * v_i  - 2*(R*P + X*Q)
#     (R/X aqui do SVR; pode ficar 0)
# ------------------------------------------------------------
def tap_onehot(mm, rid, ph, t):
    return sum(mm.ytap[rid,ph,t,k] for k in mm.TAP_K) == 1
m.SVR_OneHot = pyo.Constraint(m.REG_PH, m.T, rule=tap_onehot)

# McCormick: w = v_i * y, com y binária, v_i em [V2_MIN,V2_MAX]
def mcc1(mm, rid, ph, t, k):
    i = reg_mv_bus[rid]
    return mm.wtap[rid,ph,t,k] <= M_V2_MAX * mm.ytap[rid,ph,t,k]
def mcc2(mm, rid, ph, t, k):
    i = reg_mv_bus[rid]
    return mm.wtap[rid,ph,t,k] >= M_V2_MIN * mm.ytap[rid,ph,t,k]
def mcc3(mm, rid, ph, t, k):
    i = reg_mv_bus[rid]
    return mm.wtap[rid,ph,t,k] <= mm.v_MT[i,ph,t] - M_V2_MIN*(1-mm.ytap[rid,ph,t,k])
def mcc4(mm, rid, ph, t, k):
    i = reg_mv_bus[rid]
    return mm.wtap[rid,ph,t,k] >= mm.v_MT[i,ph,t] - M_V2_MAX*(1-mm.ytap[rid,ph,t,k])

m.SVR_Mc1 = pyo.Constraint(m.REG_PH, m.T, m.TAP_K, rule=mcc1)
m.SVR_Mc2 = pyo.Constraint(m.REG_PH, m.T, m.TAP_K, rule=mcc2)
m.SVR_Mc3 = pyo.Constraint(m.REG_PH, m.T, m.TAP_K, rule=mcc3)
m.SVR_Mc4 = pyo.Constraint(m.REG_PH, m.T, m.TAP_K, rule=mcc4)

def svr_voltage(mm, rid, ph, t):
    i = reg_mv_bus[rid]
    j = reg_lv_bus[rid]
    return mm.v_MT[j,ph,t] == sum(TAP_RATIO2[k]*mm.wtap[rid,ph,t,k] for k in mm.TAP_K) \
           - 2*(R_reg[(rid,ph)]*mm.Preg[rid,ph,t] + X_reg[(rid,ph)]*mm.Qreg[rid,ph,t])
m.SVR_V = pyo.Constraint(m.REG_PH, m.T, rule=svr_voltage)

In [315]:
# ------------------------------------------------------------
# (J) Balanços nodais (MT) incluindo trafos, SVR e fonte slack
# ------------------------------------------------------------
def pbal_mt(mm, bus, ph, t):
    inflow  = sum(mm.P_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.toMT[br]==bus)
    outflow = sum(mm.P_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.frmMT[br]==bus)

    tr_term = sum((mm.Ptr_pos[tid,ph,t]-mm.Ptr_neg[tid,ph,t]) + mm.Ploss_tr[tid,ph,t]
                  for (tid,p) in mm.TR_PH if p==ph and trafo_mv_bus[tid]==bus)

    # SVR entra em lv_bus como "inflow" e sai do mv_bus como "outflow+loss"
    reg_in  = sum(mm.Preg[rid,ph,t] for (rid,p) in mm.REG_PH if p==ph and reg_lv_bus[rid]==bus)
    reg_out = sum(mm.Preg[rid,ph,t] + mm.Ploss_reg[rid,ph,t] for (rid,p) in mm.REG_PH if p==ph and reg_mv_bus[rid]==bus)

    sub = mm.Psub[ph,t] if bus==slack_bus_mt else 0.0
    return inflow - outflow + reg_in - reg_out + sub == tr_term
m.MT_Pbal = pyo.Constraint(m.BUS_PH_MT, m.T, rule=pbal_mt)

def qbal_mt(mm, bus, ph, t):
    inflow  = sum(mm.Q_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.toMT[br]==bus)
    outflow = sum(mm.Q_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.frmMT[br]==bus)

    tr_term = sum((mm.Qtr_pos[tid,ph,t]-mm.Qtr_neg[tid,ph,t]) + mm.Qloss_tr[tid,ph,t]
                  for (tid,p) in mm.TR_PH if p==ph and trafo_mv_bus[tid]==bus)

    reg_in  = sum(mm.Qreg[rid,ph,t] for (rid,p) in mm.REG_PH if p==ph and reg_lv_bus[rid]==bus)
    reg_out = sum(mm.Qreg[rid,ph,t] + mm.Qloss_reg[rid,ph,t] for (rid,p) in mm.REG_PH if p==ph and reg_mv_bus[rid]==bus)

    sub = mm.Qsub[ph,t] if bus==slack_bus_mt else 0.0
    return inflow - outflow + reg_in - reg_out + sub == tr_term
m.MT_Qbal = pyo.Constraint(m.BUS_PH_MT, m.T, rule=qbal_mt)

In [316]:
# ------------------------------------------------------------
# (K) Slack: fixa tensão no slack MT
# ------------------------------------------------------------
for ph in PHASES:
    for t in T_LIST:
        m.v_MT[slack_bus_mt, ph, t].fix(1.0)

# ------------------------------------------------------------
# (L) Tap operation counting (opcional)
#     - Definimos "posição tap" implícita via ytap
#     - Para contar operação, criamos variável tap_pos = sum k*y
#       e |tap_pos(t)-tap_pos(t-1)| via binários up/dn
# ------------------------------------------------------------
m.tap_pos = pyo.Var(m.REG_PH, m.T, domain=pyo.NonNegativeReals)

def tap_pos_def(mm, rid, ph, t):
    return mm.tap_pos[rid,ph,t] == sum(k*mm.ytap[rid,ph,t,k] for k in mm.TAP_K)
m.TapPosDef = pyo.Constraint(m.REG_PH, m.T, rule=tap_pos_def)

t0, prev_t = build_prev_map(T_LIST)

# inicia tap no primeiro instante (0)
for (rid,ph) in REG_PH:
    m.tap_pos[rid,ph,t0].fix(0.0)

def tap_evol_updn(mm, rid, ph, t):
    if t == t0:
        return pyo.Constraint.Skip
    tp = prev_t[t]
    # tap_pos(t) - tap_pos(tp) = up - dn
    return mm.tap_pos[rid,ph,t] - mm.tap_pos[rid,ph,tp] == mm.tap_up[rid,ph,t] - mm.tap_dn[rid,ph,t]
m.TapEvol = pyo.Constraint(m.REG_PH, m.T, rule=tap_evol_updn)


In [317]:
# ------------------------------------------------------------
# 5) OBJETIVO
# - Minimiza número de operações tap (up+dn)
# - + termo pequeno para perdas MT (opcional)
# ------------------------------------------------------------
m.obj = pyo.Objective(
    expr = sum(m.tap_up[rid,ph,t] + m.tap_dn[rid,ph,t] for (rid,ph) in m.REG_PH for t in m.T if t!=t0)
         + 1e-4*sum(m.Ploss_MT[br,ph,t] for (br,ph) in m.BR_PH_MT for t in m.T),
    sense = pyo.minimize
)

In [318]:
# ============================================================
# 6) SOLVE (HiGHS MILP)
# ============================================================
solver = pyo.SolverFactory("highs")
res = solver.solve(m, tee=True)
print(res.solver.termination_condition)

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 329853 rows; 322170 cols; 945390 nonzeros; 4602 integer variables (4602 binary)
Coefficient ranges:
  Matrix  [2e-04, 3e+03]
  Cost    [1e-04, 1e+00]
  Bound   [9e-01, 1e+00]
  RHS     [2e-04, 3e+03]
Presolving model
192954 rows, 234216 cols, 619092 nonzeros  1s
137392 rows, 174736 cols, 603755 nonzeros  2s
Presolve: Infeasible

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol        

NoFeasibleSolutionError: A feasible solution was not found, so no solution can be loaded. Please set opt.config.load_solutions=False and check results.solution_status and results.incumbent_objective before loading a solution.